This notebook reads in the cleaned OSC file (1.OSCCleaned.csv') and searches through PHANGS CO map images, looking for OSC objects that have gone off in the ALMA footprint. 

It then converts the ra & dec coordinates to map pixels and measures the pixel intensity there.

This notebook prints the OSC Object data on the '2.SNeCOMasterCat.txt'. 

This notebook then turns the 2.SNeCOMasterCat.txt file into a dataframe, where the non SN objects are removed and then the cleaned table is printed to '2.SNeCOMasterCatClean.csv'.

This cleaned table is used to plot all of the galaxy images and their respective SNe in ../Figures/2.AllGalaxyImages & 2.SNeGalaxyImages.

In [62]:
#import packages and formatting statements

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.io import ascii

plt.rc('text',usetex=False)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [14]:
#OPEN CANDIDATE CATALOG AND READ IN RAS, DECS, NAMES
galaxies = np.loadtxt('../Data/0.PHANGSGalaxies_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
images = np.loadtxt('../Data/0.PHANGSGalaxiesImg_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
errFiles = np.loadtxt('../Data/0.PHANGSGalaxiesImgErr_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
ewFiles = np.loadtxt('../Data/0.PHANGSGalaxiesEW_12_7_tp.txt', dtype = str, unpack=True, usecols = 0)
SNe = np.loadtxt('../Data/1.OSCRaDeg.txt', dtype = str, unpack=True, usecols = 0)
ras = np.loadtxt('../Data/1.OSCRaDeg.txt',delimiter='" ', unpack = True, usecols=1)
decs = np.loadtxt('../Data/1.OSCDecDeg.txt',delimiter='" ',unpack = True, usecols = (1))
types = np.loadtxt('../Data/1.OSCCleaned.csv', dtype=str, unpack=True, usecols=3)

In [15]:
#CHECK IF RAS AND DECS ARE IN AN IMAGE
def check_in_image(SNras, SNdecs, SNnames, SNtypes, GalaxyImg, ImgErr, EW):
    hdulist = pyfits.open(GalaxyImg)
    hdulist2 = pyfits.open(ImgErr)
    hdulist3 = pyfits.open(EW)
    map = hdulist[0].data
    map2 = hdulist2[0].data
    map3 = hdulist3[0].data
    wcs = WCS(hdulist[0].header, naxis=2)
    beamMaj = hdulist[0].header['BMAJ']
    beamMin = hdulist[0].header['BMIN']
    if(beamMaj==beamMin):
        beamSize = beamMaj
    else:
        beamSize = 0.0
    coords_arr = np.column_stack((SNras, SNdecs)) # ras and decs now [ra,dec]
    pix_x, pix_y = wcs.wcs_world2pix(SNras,SNdecs,0)

    #use world coordinates of all SNe to see if any fall in image (our version of footprint_contains)
    naxis = wcs._naxis #size of image
    naxis1 = hdulist[0].header['NAXIS1']
    naxis2 = hdulist[0].header['NAXIS2']
    is_in_x = (pix_x >= 0) & (pix_x <= naxis[0]-1) #because of 0-indexing
    is_in_y = (pix_y >= 0) & (pix_y <= naxis[1]-1)
             
    #get the name, ra, and dec of the SNe that fall in image
    #boolean array indexing (gives back array of Trues and Falses)
    #we are pulling out the SNe that are True and assigning them to own arrays
    name_in_image = np.array(SNnames)[is_in_x & is_in_y]
    type_in_image = np.array(SNtypes)[is_in_x & is_in_y]
    ra_in_image = np.array(SNras)[is_in_x & is_in_y]
    dec_in_image = np.array(SNdecs)[is_in_x & is_in_y]

    x_coord = np.array(pix_x)[is_in_x & is_in_y]
    y_coord = np.array(pix_y)[is_in_x & is_in_y]
    
    return (name_in_image, type_in_image, ra_in_image, dec_in_image, x_coord, y_coord, map, map2, map3, naxis1, naxis2, beamSize)

In [16]:
# Master Catalogue File That has all OSC entries within PHANGS-ALMA footprint
# Also contains LRN

with open('../Data/2.SNeCOMasterCat.txt', 'w') as text_file:
    print('#Galaxy,Supernova,Ra,Dec,xCoord,yCoord,Type,Intensity(12m+7m+tp),Err,EW,beamSize', file = text_file)

for i in range(len(galaxies)):
    found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, map2,map3, naxis1, naxis2, beamSize = check_in_image(ras, decs, SNe, types, images[i], errFiles[i], ewFiles[i])
 
    intensity = []
    intensityErr = []
    equivalentWidth = []
    equivalentWidthErr = []
    
    for k in range(len(found_type)):

        xVal = int(round(x_coord[k]))
        yVal = int(round(y_coord[k]))
        if (yVal <= naxis2) and (xVal <= naxis1):
            intensity.append(map[yVal, xVal])
            intensityErr.append(map2[yVal, xVal])
            equivalentWidth.append(map3[yVal, xVal])

        else: 
            intensity.append(np.nan)
            intensityErr.append(np.nan)
            equivalentWidth.append(np.nan)  

    with open('../Data/2.SNeCOMasterCat.txt', 'a') as text_file:
        for j in range(len(found_name)):
            print(galaxies[i], ",", found_name[j],",", found_ra[j], ",",found_dec[j],",", x_coord[j],",",y_coord[j],",", found_type[j],",", intensity[j],",", intensityErr[j],",",equivalentWidth[j],",",beamSize, file = text_file)


FileNotFoundError: [Errno 2] No such file or directory: '/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_broad_mom0.fits'

In [66]:
# Turn file into dataStructure and clean

dataFile = '../Data/2.SNeCOMasterCat.txt'
galaxy, SN_name, ra, dec, xVal, yVal, SN_type, co_21_int, co_21_err, co_21_ew, beamsize  = np.genfromtxt(dataFile,  dtype=str, unpack=True, delimiter = ',')

SN_name = [x[2:-2] for x in SN_name]
ra   = [float(x) for x in ra]
ra   = [round(x,4) for x in ra]
dec  = [float(x) for x in dec]
dec  = [round(x,4) for x in dec]

data = Table({'Galaxy'    : galaxy,
              'SN Name'  : SN_name,
              'SN Type'  : SN_type,
              'ra'       : ra,
              'Dec'      : dec,
              'xCoord'   : xVal,
              'yCoord'   : yVal,
              'co21int'  : co_21_int,
              'co21err'  : co_21_err,
              'co21ew'   : co_21_ew,
              'beamsize' : beamsize,
             },
             names=['Galaxy', 'SN Name','SN Type','ra','Dec','xCoord', 'yCoord', 'co21int', 'co21err', "co21ew", 'beamsize'])


# Here we are going to filter out our undesired OSC entries:
    # One is a double counted SN named "PTSS-19clju"
    # The other is a LRN

flags = []
flag1 = data['SN Name'] == "PTSS-19clju"

flag2 = []

for i in range(len(SN_type)):
    if "LRN" in data['SN Type'][i]:
        flag2.append(True)
    else:
        flag2.append(False)

for i in range(len(flag1)):
    if flag1[i] == True or flag2[i] == True:
        flags.append(True)
    else:
        flags.append(False)
        
flagsArr = np.array(flags)
cleanData = data[flagsArr==False]

cleanData

ascii.write(cleanData, '../Data/2.SNeCOMasterCatClean.csv', format='csv', fast_writer=False)  

Galaxy,SN Name,SN Type,ra,Dec,xCoord,yCoord,co21int,co21err,co21ew,beamsize
str21,str11,str21,float64,float64,str21,str21,str21,str21,str21,str21
circinus,AT2020juh,Candidate,213.3379,-65.3413,51.93785788436541,92.5424301237033,4.5456676,1.311794,nan,0.002032163908078
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078
ngc0247,AT2019npi,Candidate,11.7453,-20.7083,175.7229173476427,311.8359696835978,0.0,0.10829854,nan,0.002364257174682
ngc0253,AT2019npd,Candidate,11.7363,-25.3768,527.82046247017,86.1134265201371,1.7295623,0.37351817,1.8044568,0.00232543637022
ngc0253,AT2020hol,Candidate,11.8539,-25.3571,336.54099737127774,121.66030472885404,0.23782109,0.63490534,nan,0.00232543637022
ngc0253,AT2019pck,Candidate,11.8514,-25.2261,340.64395858591797,357.55919248119864,nan,nan,nan,0.00232543637022
ngc0253,SN1940E,I,11.8783,-25.2934,296.9010534051223,236.26662905273994,218.98796,1.4441332,45.862267,0.00232543637022
ngc0300,SN2019qyl,IIn/LBV,13.7399,-37.6444,124.97990204934794,202.45166439134732,-0.5273654,0.45072383,nan,0.002271061982879
ngc0628,SN2013ej,II,24.2007,15.7586,333.46022572413716,356.77980642079694,0.9655536,1.4078557,nan,0.0003114595189268


In [72]:
# Recreate Galaxy Files That have only cleaned objects

data = ascii.read('../Data/2.SNeCOMasterCatClean.csv')

for i in range(len(galaxies)):
    
    found_name, found_type, found_ra, found_dec, x_coord, y_coord, map, map2, map3, naxis1, naxis2, beamSize = check_in_image(data['ra'], data['Dec'], data['SN Name'], data['SN Type'], images[i], errFiles[i], ewFiles[i])
    
    plt.figure(figsize=(7, 7))
    logmap = np.log10(map)
    plt.imshow(logmap, origin='lower', interpolation='nearest', zorder=1)
    
    for k in range(len(found_name)):

        string = found_name[k]
        
        if (len(found_type[k])>1):
            if found_type[k][1] == "I":
                #colorCode = 'chartreuse'
                colorCode = 'black'
                mkr = '*'
            elif found_type[k][1] == "b" or found_type[k][1] == "c":
                #colorCode = 'salmon'
                colorCode = 'darkblue'
                mkr = '^'
            elif found_type[k][1] == "a":
                #colorCode = 'deeppink'
                colorCode = 'indigo'
                mkr = 'o'
            else:
                colorCode = 'slategrey'
                mkr = ''
        else:
            #colorCode = 'cyan'
            colorCode = 'maroon'
            mkr='s'
            
        plt.plot(x_coord[k], y_coord[k], marker=mkr, ms=15, color=colorCode, zorder=10)
        plt.text(x_coord[k]-20, y_coord[k]+8, string, fontsize=16, fontweight='bold', color=colorCode)
        
    plt.axis('off')
    plt.title(galaxies[i], fontsize=24, fontweight='bold')
    plt.tight_layout()
    plt.savefig('../Figures/2.AllGalaxyImages/' + galaxies[i] + '.png')
    if (len(found_name) > 0):
        plt.savefig('../Figures/2.SNeGalaxyImages/' + galaxies[i] + '.png')
    plt.close()

Galaxy,SN Name,SN Type,ra,Dec,xCoord,yCoord,co21int,co21err,co21ew,beamsize
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64
circinus,AT2020juh,Candidate,213.3379,-65.3413,51.93785788436541,92.5424301237033,4.5456676,1.311794,nan,0.002032163908078
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078
ngc0247,AT2019npi,Candidate,11.7453,-20.7083,175.7229173476427,311.8359696835978,0.0,0.10829854,nan,0.002364257174682
ngc0253,AT2019npd,Candidate,11.7363,-25.3768,527.82046247017,86.1134265201371,1.7295623,0.37351817,1.8044568,0.00232543637022
ngc0253,AT2020hol,Candidate,11.8539,-25.3571,336.54099737127774,121.66030472885404,0.23782109,0.63490534,nan,0.00232543637022
ngc0253,AT2019pck,Candidate,11.8514,-25.2261,340.64395858591797,357.55919248119864,nan,nan,nan,0.00232543637022
ngc0253,SN1940E,I,11.8783,-25.2934,296.9010534051223,236.26662905273994,218.98796,1.4441332,45.862267,0.00232543637022
ngc0300,SN2019qyl,IIn/LBV,13.7399,-37.6444,124.97990204934794,202.45166439134732,-0.5273654,0.45072383,nan,0.002271061982879
ngc0628,SN2013ej,II,24.2007,15.7586,333.46022572413716,356.77980642079694,0.9655536,1.4078557,nan,0.0003114595189268
